# Phase 7: Comprehensive Comparison

**Quantum-Enhanced Simulation Learning for Reinforcement Learning**

Author: Saurabh Jalendra  
Institution: BITS Pilani (WILP Division)  
Date: November 2025

---

## Overview

This notebook provides a **comprehensive, statistically rigorous comparison** of all
implemented quantum-inspired world model training approaches:

### Approaches Compared

1. **Classical Baseline**: Standard DreamerV3-style world model
2. **QAOA-Enhanced**: Quantum approximate optimization-inspired training
3. **Superposition Replay**: Quantum superposition-inspired experience replay
4. **Gate-Enhanced**: Quantum gate-inspired neural network layers
5. **Error Correction**: Quantum error correction-inspired ensemble

### Evaluation Metrics

- **Sample Efficiency**: Steps to reach target performance
- **Training Speed**: Wall-clock time to convergence
- **Prediction Accuracy**: MSE on test trajectories
- **Robustness**: Performance under noise

### Statistical Analysis

- Multiple seeds (5 per configuration)
- Mann-Whitney U tests
- Cohen's d effect size
- 95% confidence intervals

---

## 7.1 Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import math
import time
from typing import Dict, List, Tuple, Optional, Callable
from dataclasses import dataclass, field
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import gymnasium as gym

from src.utils import set_seed, get_device, MetricLogger, Timer, COLORS

# Set seed for reproducibility
set_seed(42)
device = get_device()
print(f"Using device: {device}")

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

## 7.2 Model Implementations

Import all model implementations from previous phases.

In [ ]:
# Base World Model (shared architecture for fair comparison)
class BaseWorldModel(nn.Module):
    """
    Base world model architecture used by all approaches.
    
    This ensures fair comparison by using identical architectures,
    only varying the training procedure or enhancements.
    """
    
    def __init__(
        self,
        obs_dim: int,
        action_dim: int,
        hidden_dim: int = 256,
        deter_dim: int = 128,
        stoch_dim: int = 32
    ):
        super().__init__()
        self.obs_dim = obs_dim
        self.action_dim = action_dim
        self.deter_dim = deter_dim
        self.stoch_dim = stoch_dim
        self.state_dim = deter_dim + stoch_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(obs_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ELU()
        )
        
        # RSSM
        self.gru = nn.GRUCell(stoch_dim + action_dim, deter_dim)
        
        self.prior = nn.Sequential(
            nn.Linear(deter_dim, hidden_dim // 2),
            nn.ELU(),
            nn.Linear(hidden_dim // 2, stoch_dim * 2)
        )
        
        self.posterior = nn.Sequential(
            nn.Linear(deter_dim + hidden_dim // 2, hidden_dim // 2),
            nn.ELU(),
            nn.Linear(hidden_dim // 2, stoch_dim * 2)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(self.state_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, obs_dim * 2)
        )
        
        # Reward predictor
        self.reward_pred = nn.Sequential(
            nn.Linear(self.state_dim, hidden_dim // 2),
            nn.ELU(),
            nn.Linear(hidden_dim // 2, 1)
        )
    
    def initial_state(self, batch_size: int, device: torch.device) -> Dict[str, Tensor]:
        return {
            'deter': torch.zeros(batch_size, self.deter_dim, device=device),
            'stoch': torch.zeros(batch_size, self.stoch_dim, device=device)
        }
    
    def forward(self, obs_seq: Tensor, action_seq: Tensor) -> Dict[str, Tensor]:
        batch_size, seq_len = obs_seq.shape[:2]
        device = obs_seq.device
        
        state = self.initial_state(batch_size, device)
        
        states, prior_means, prior_stds = [], [], []
        post_means, post_stds = [], []
        
        for t in range(seq_len):
            embed = self.encoder(obs_seq[:, t])
            gru_input = torch.cat([state['stoch'], action_seq[:, t]], dim=-1)
            deter = self.gru(gru_input, state['deter'])
            
            # Prior
            prior_stats = self.prior(deter)
            prior_mean, prior_log_std = torch.chunk(prior_stats, 2, dim=-1)
            prior_std = F.softplus(prior_log_std) + 0.1
            
            # Posterior
            post_input = torch.cat([deter, embed], dim=-1)
            post_stats = self.posterior(post_input)
            post_mean, post_log_std = torch.chunk(post_stats, 2, dim=-1)
            post_std = F.softplus(post_log_std) + 0.1
            
            stoch = post_mean + post_std * torch.randn_like(post_std)
            state = {'deter': deter, 'stoch': stoch}
            
            full_state = torch.cat([deter, stoch], dim=-1)
            states.append(full_state)
            prior_means.append(prior_mean)
            prior_stds.append(prior_std)
            post_means.append(post_mean)
            post_stds.append(post_std)
        
        states = torch.stack(states, dim=1)
        flat_states = states.reshape(-1, self.state_dim)
        
        dec_output = self.decoder(flat_states)
        obs_mean, obs_log_std = torch.chunk(dec_output, 2, dim=-1)
        obs_mean = obs_mean.reshape(batch_size, seq_len, -1)
        obs_log_std = obs_log_std.clamp(-10, 2).reshape(batch_size, seq_len, -1)
        
        reward_pred = self.reward_pred(flat_states).reshape(batch_size, seq_len)
        
        return {
            'states': states,
            'obs_mean': obs_mean,
            'obs_log_std': obs_log_std,
            'reward_pred': reward_pred,
            'prior_mean': torch.stack(prior_means, dim=1),
            'prior_std': torch.stack(prior_stds, dim=1),
            'post_mean': torch.stack(post_means, dim=1),
            'post_std': torch.stack(post_stds, dim=1)
        }

In [ ]:
# Training approaches

class BaselineTrainer:
    """Standard baseline trainer with Adam optimizer."""
    
    def __init__(self, model, lr=1e-4, kl_weight=1.0, free_nats=3.0):
        self.model = model
        self.kl_weight = kl_weight
        self.free_nats = free_nats
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
    
    def compute_loss(self, obs, actions, rewards, outputs):
        obs_dist = torch.distributions.Normal(
            outputs['obs_mean'], torch.exp(outputs['obs_log_std'])
        )
        recon_loss = -obs_dist.log_prob(obs).mean()
        
        prior_dist = torch.distributions.Normal(outputs['prior_mean'], outputs['prior_std'])
        post_dist = torch.distributions.Normal(outputs['post_mean'], outputs['post_std'])
        kl_loss = torch.distributions.kl_divergence(post_dist, prior_dist).mean()
        kl_loss = torch.maximum(kl_loss, torch.tensor(self.free_nats, device=kl_loss.device))
        
        reward_loss = F.mse_loss(outputs['reward_pred'], rewards)
        
        return recon_loss + self.kl_weight * kl_loss + reward_loss
    
    def train_step(self, obs, actions, rewards):
        self.model.train()
        self.optimizer.zero_grad()
        
        outputs = self.model(obs, actions)
        loss = self.compute_loss(obs, actions, rewards, outputs)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 100.0)
        self.optimizer.step()
        
        return {'loss': loss.item()}


class QAOATrainer:
    """QAOA-inspired trainer with alternating cost/mixing operators."""
    
    def __init__(self, model, lr=1e-4, gamma=0.5, beta=0.1, p=3):
        self.model = model
        self.gamma = gamma
        self.beta = beta
        self.p = p
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
        self.step_count = 0
    
    def compute_loss(self, obs, actions, rewards, outputs):
        obs_dist = torch.distributions.Normal(
            outputs['obs_mean'], torch.exp(outputs['obs_log_std'])
        )
        recon_loss = -obs_dist.log_prob(obs).mean()
        
        prior_dist = torch.distributions.Normal(outputs['prior_mean'], outputs['prior_std'])
        post_dist = torch.distributions.Normal(outputs['post_mean'], outputs['post_std'])
        kl_loss = torch.distributions.kl_divergence(post_dist, prior_dist).mean()
        kl_loss = torch.maximum(kl_loss, torch.tensor(3.0, device=kl_loss.device))
        
        reward_loss = F.mse_loss(outputs['reward_pred'], rewards)
        
        return recon_loss + kl_loss + reward_loss
    
    def train_step(self, obs, actions, rewards):
        self.model.train()
        self.step_count += 1
        
        # Alternate between cost and mixing phases
        phase = self.step_count % (2 * self.p)
        is_mixing = (phase % 2 == 1)
        
        # Dynamic gamma and beta scheduling
        decay = 0.99 ** (self.step_count // 100)
        gamma = self.gamma * decay
        beta = self.beta * decay
        
        self.optimizer.zero_grad()
        outputs = self.model(obs, actions)
        loss = self.compute_loss(obs, actions, rewards, outputs)
        loss.backward()
        
        if is_mixing:
            # Mixing operator: add exploration noise
            with torch.no_grad():
                for param in self.model.parameters():
                    if param.grad is not None:
                        noise = torch.randn_like(param) * beta
                        param.grad.add_(noise)
        else:
            # Cost operator: scale gradients
            with torch.no_grad():
                for param in self.model.parameters():
                    if param.grad is not None:
                        param.grad.mul_(gamma)
        
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 100.0)
        self.optimizer.step()
        
        return {'loss': loss.item()}


class SuperpositionReplayTrainer:
    """Trainer with superposition-inspired experience replay."""
    
    def __init__(self, model, lr=1e-4, alpha=0.6, beta_start=0.4):
        self.model = model
        self.alpha = alpha
        self.beta = beta_start
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
        self.priorities = None
        self.step_count = 0
    
    def compute_amplitudes(self, td_errors: Tensor) -> Tensor:
        """Compute quantum-inspired amplitudes for prioritization."""
        priorities = (td_errors.abs() + 1e-6) ** self.alpha
        amplitudes = priorities / priorities.sum()
        return amplitudes
    
    def compute_loss(self, obs, actions, rewards, outputs, weights=None):
        obs_dist = torch.distributions.Normal(
            outputs['obs_mean'], torch.exp(outputs['obs_log_std'])
        )
        recon_loss = -obs_dist.log_prob(obs)
        
        prior_dist = torch.distributions.Normal(outputs['prior_mean'], outputs['prior_std'])
        post_dist = torch.distributions.Normal(outputs['post_mean'], outputs['post_std'])
        kl_loss = torch.distributions.kl_divergence(post_dist, prior_dist)
        
        reward_loss = (outputs['reward_pred'] - rewards) ** 2
        
        if weights is not None:
            weights = weights.unsqueeze(-1)  # (batch, 1)
            recon_loss = (recon_loss * weights.unsqueeze(-1)).mean()
            kl_loss = (kl_loss * weights.unsqueeze(-1)).mean()
            reward_loss = (reward_loss * weights).mean()
        else:
            recon_loss = recon_loss.mean()
            kl_loss = kl_loss.mean()
            reward_loss = reward_loss.mean()
        
        kl_loss = torch.maximum(kl_loss, torch.tensor(3.0, device=kl_loss.device))
        
        return recon_loss + kl_loss + reward_loss, reward_loss
    
    def train_step(self, obs, actions, rewards):
        self.model.train()
        self.step_count += 1
        
        # Anneal beta
        self.beta = min(1.0, 0.4 + self.step_count * 0.001)
        
        # First pass: compute TD errors
        with torch.no_grad():
            outputs = self.model(obs, actions)
            td_errors = (outputs['reward_pred'] - rewards).abs().mean(dim=-1)  # (batch,)
            amplitudes = self.compute_amplitudes(td_errors)
        
        # Importance sampling weights
        N = obs.shape[0]
        weights = (N * amplitudes) ** (-self.beta)
        weights = weights / weights.max()
        
        # Training pass with weighted loss
        self.optimizer.zero_grad()
        outputs = self.model(obs, actions)
        loss, _ = self.compute_loss(obs, actions, rewards, outputs, weights)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 100.0)
        self.optimizer.step()
        
        return {'loss': loss.item()}

In [ ]:
# Gate-Enhanced Model

class QuantumGateLayer(nn.Module):
    """Combined quantum gate-inspired layer."""
    
    def __init__(self, dim: int):
        super().__init__()
        self.dim = dim
        
        # Rotation angles
        num_rotations = dim // 2
        self.angles = nn.Parameter(torch.randn(num_rotations, 3) * 0.1)
        indices = torch.randperm(dim)[:num_rotations * 2]
        self.register_buffer('idx1', indices[:num_rotations])
        self.register_buffer('idx2', indices[num_rotations:])
        
        # Phase
        self.phases = nn.Parameter(torch.randn(dim) * 0.1)
        self.amplitude = nn.Parameter(torch.ones(dim))
        
        # Layer norm
        self.norm = nn.LayerNorm(dim)
    
    def forward(self, x: Tensor) -> Tensor:
        # Rotation
        y = x.clone()
        x1, x2 = x[:, self.idx1], x[:, self.idx2]
        
        for i in range(3):
            cos_t = torch.cos(self.angles[:, i])
            sin_t = torch.sin(self.angles[:, i])
            x1_new = cos_t * x1 - sin_t * x2
            x2_new = sin_t * x1 + cos_t * x2
            x1, x2 = x1_new, x2_new
        
        y = y.scatter(1, self.idx1.unsqueeze(0).expand(x.shape[0], -1), x1)
        y = y.scatter(1, self.idx2.unsqueeze(0).expand(x.shape[0], -1), x2)
        
        # Phase
        cos_p = torch.cos(self.phases)
        sin_p = torch.sin(self.phases)
        y = self.amplitude * (y * cos_p + torch.abs(y) * sin_p)
        
        return self.norm(x + y)


class GateEnhancedModel(nn.Module):
    """World model with quantum gate-enhanced layers."""
    
    def __init__(self, obs_dim, action_dim, hidden_dim=256, deter_dim=128, stoch_dim=32):
        super().__init__()
        self.obs_dim = obs_dim
        self.action_dim = action_dim
        self.deter_dim = deter_dim
        self.stoch_dim = stoch_dim
        self.state_dim = deter_dim + stoch_dim
        
        # Gate-enhanced encoder
        self.encoder = nn.Sequential(
            nn.Linear(obs_dim, hidden_dim),
            nn.ELU(),
            QuantumGateLayer(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ELU()
        )
        
        self.gru = nn.GRUCell(stoch_dim + action_dim, deter_dim)
        
        self.prior = nn.Sequential(
            nn.Linear(deter_dim, hidden_dim // 2),
            nn.ELU(),
            QuantumGateLayer(hidden_dim // 2),
            nn.Linear(hidden_dim // 2, stoch_dim * 2)
        )
        
        self.posterior = nn.Sequential(
            nn.Linear(deter_dim + hidden_dim // 2, hidden_dim // 2),
            nn.ELU(),
            QuantumGateLayer(hidden_dim // 2),
            nn.Linear(hidden_dim // 2, stoch_dim * 2)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.state_dim, hidden_dim),
            nn.ELU(),
            QuantumGateLayer(hidden_dim),
            nn.Linear(hidden_dim, obs_dim * 2)
        )
        
        self.reward_pred = nn.Sequential(
            nn.Linear(self.state_dim, hidden_dim // 2),
            nn.ELU(),
            nn.Linear(hidden_dim // 2, 1)
        )
    
    def initial_state(self, batch_size, device):
        return {
            'deter': torch.zeros(batch_size, self.deter_dim, device=device),
            'stoch': torch.zeros(batch_size, self.stoch_dim, device=device)
        }
    
    def forward(self, obs_seq, action_seq):
        batch_size, seq_len = obs_seq.shape[:2]
        device = obs_seq.device
        
        state = self.initial_state(batch_size, device)
        states, prior_means, prior_stds, post_means, post_stds = [], [], [], [], []
        
        for t in range(seq_len):
            embed = self.encoder(obs_seq[:, t])
            gru_input = torch.cat([state['stoch'], action_seq[:, t]], dim=-1)
            deter = self.gru(gru_input, state['deter'])
            
            prior_stats = self.prior(deter)
            prior_mean, prior_log_std = torch.chunk(prior_stats, 2, dim=-1)
            prior_std = F.softplus(prior_log_std) + 0.1
            
            post_input = torch.cat([deter, embed], dim=-1)
            post_stats = self.posterior(post_input)
            post_mean, post_log_std = torch.chunk(post_stats, 2, dim=-1)
            post_std = F.softplus(post_log_std) + 0.1
            
            stoch = post_mean + post_std * torch.randn_like(post_std)
            state = {'deter': deter, 'stoch': stoch}
            
            states.append(torch.cat([deter, stoch], dim=-1))
            prior_means.append(prior_mean)
            prior_stds.append(prior_std)
            post_means.append(post_mean)
            post_stds.append(post_std)
        
        states = torch.stack(states, dim=1)
        flat_states = states.reshape(-1, self.state_dim)
        
        dec_output = self.decoder(flat_states)
        obs_mean, obs_log_std = torch.chunk(dec_output, 2, dim=-1)
        obs_mean = obs_mean.reshape(batch_size, seq_len, -1)
        obs_log_std = obs_log_std.clamp(-10, 2).reshape(batch_size, seq_len, -1)
        
        reward_pred = self.reward_pred(flat_states).reshape(batch_size, seq_len)
        
        return {
            'states': states, 'obs_mean': obs_mean, 'obs_log_std': obs_log_std,
            'reward_pred': reward_pred, 'prior_mean': torch.stack(prior_means, dim=1),
            'prior_std': torch.stack(prior_stds, dim=1),
            'post_mean': torch.stack(post_means, dim=1),
            'post_std': torch.stack(post_stds, dim=1)
        }

In [ ]:
# Error Correction Ensemble

class EnsembleModel(nn.Module):
    """Ensemble with error correction."""
    
    def __init__(self, obs_dim, action_dim, num_models=5, hidden_dim=128, 
                 deter_dim=64, stoch_dim=16):
        super().__init__()
        self.num_models = num_models
        
        self.models = nn.ModuleList([
            BaseWorldModel(obs_dim, action_dim, hidden_dim, deter_dim, stoch_dim)
            for _ in range(num_models)
        ])
        
        # Initialize with different seeds
        for i, model in enumerate(self.models):
            torch.manual_seed(42 + i * 1000)
            for m in model.modules():
                if hasattr(m, 'reset_parameters'):
                    m.reset_parameters()
    
    def forward(self, obs_seq, action_seq):
        all_outputs = [model(obs_seq, action_seq) for model in self.models]
        
        # Weighted averaging based on disagreement
        obs_means = torch.stack([o['obs_mean'] for o in all_outputs], dim=0)
        ensemble_mean = obs_means.mean(dim=0)
        deviations = (obs_means - ensemble_mean.unsqueeze(0)).abs().mean(dim=(2, 3))
        weights = 1.0 / (deviations + 1e-8)
        weights = weights / weights.sum(dim=0, keepdim=True)
        
        # Weighted prediction
        obs_mean = (obs_means * weights.unsqueeze(-1).unsqueeze(-1)).sum(dim=0)
        
        obs_log_stds = torch.stack([o['obs_log_std'] for o in all_outputs], dim=0)
        obs_log_std = (obs_log_stds * weights.unsqueeze(-1).unsqueeze(-1)).sum(dim=0)
        
        reward_preds = torch.stack([o['reward_pred'] for o in all_outputs], dim=0)
        reward_pred = (reward_preds * weights.unsqueeze(-1)).sum(dim=0)
        
        # Average stats for loss computation
        return {
            'obs_mean': obs_mean,
            'obs_log_std': obs_log_std,
            'reward_pred': reward_pred,
            'prior_mean': torch.stack([o['prior_mean'] for o in all_outputs]).mean(0),
            'prior_std': torch.stack([o['prior_std'] for o in all_outputs]).mean(0),
            'post_mean': torch.stack([o['post_mean'] for o in all_outputs]).mean(0),
            'post_std': torch.stack([o['post_std'] for o in all_outputs]).mean(0),
            'states': torch.stack([o['states'] for o in all_outputs]).mean(0),
            'all_outputs': all_outputs
        }

## 7.3 Experiment Framework

Framework for running experiments across multiple seeds and environments.

In [ ]:
@dataclass
class ExperimentConfig:
    """Configuration for a single experiment."""
    name: str
    env_name: str
    num_seeds: int = 5
    num_episodes: int = 20
    num_epochs: int = 50
    batch_size: int = 32
    seq_len: int = 20
    learning_rate: float = 1e-4


@dataclass
class ExperimentResult:
    """Results from a single experiment run."""
    approach: str
    seed: int
    final_loss: float
    training_time: float
    loss_history: List[float]
    prediction_error: float
    

def collect_data(env_name: str, num_episodes: int = 20, max_steps: int = 200):
    """Collect episodes from environment."""
    env = gym.make(env_name)
    episodes = []
    
    for _ in range(num_episodes):
        obs_list, action_list, reward_list = [], [], []
        obs, _ = env.reset()
        obs_list.append(obs)
        
        for _ in range(max_steps):
            action = env.action_space.sample()
            next_obs, reward, terminated, truncated, _ = env.step(action)
            
            if isinstance(action, (int, np.integer)):
                action_list.append([float(action)])
            else:
                action_list.append(action)
            reward_list.append(reward)
            obs_list.append(next_obs)
            
            if terminated or truncated:
                break
            obs = next_obs
        
        obs_list = obs_list[:-1]
        if len(obs_list) > 10:
            episodes.append({
                'obs': np.array(obs_list, dtype=np.float32),
                'actions': np.array(action_list, dtype=np.float32),
                'rewards': np.array(reward_list, dtype=np.float32)
            })
    
    env.close()
    return episodes


def create_batches(episodes, batch_size=32, seq_len=20):
    """Create training batches from episodes."""
    sequences = []
    for ep in episodes:
        ep_len = len(ep['obs'])
        for start in range(0, ep_len - seq_len, seq_len // 2):
            sequences.append({
                'obs': ep['obs'][start:start+seq_len],
                'actions': ep['actions'][start:start+seq_len],
                'rewards': ep['rewards'][start:start+seq_len]
            })
    
    np.random.shuffle(sequences)
    batches = []
    
    for i in range(0, len(sequences) - batch_size, batch_size):
        batch_seqs = sequences[i:i+batch_size]
        batches.append((
            torch.tensor(np.stack([s['obs'] for s in batch_seqs]), dtype=torch.float32, device=device),
            torch.tensor(np.stack([s['actions'] for s in batch_seqs]), dtype=torch.float32, device=device),
            torch.tensor(np.stack([s['rewards'] for s in batch_seqs]), dtype=torch.float32, device=device)
        ))
    
    return batches

In [ ]:
def run_experiment(
    config: ExperimentConfig,
    approach: str,
    seed: int,
    batches: List,
    obs_dim: int,
    action_dim: int
) -> ExperimentResult:
    """
    Run a single experiment with given configuration.
    
    Parameters
    ----------
    config : ExperimentConfig
        Experiment configuration
    approach : str
        Name of the approach
    seed : int
        Random seed
    batches : List
        Training batches
    obs_dim : int
        Observation dimension
    action_dim : int
        Action dimension
    
    Returns
    -------
    ExperimentResult
        Results from this run
    """
    set_seed(seed)
    
    # Create model and trainer based on approach
    if approach == 'baseline':
        model = BaseWorldModel(obs_dim, action_dim).to(device)
        trainer = BaselineTrainer(model, lr=config.learning_rate)
    elif approach == 'qaoa':
        model = BaseWorldModel(obs_dim, action_dim).to(device)
        trainer = QAOATrainer(model, lr=config.learning_rate)
    elif approach == 'superposition':
        model = BaseWorldModel(obs_dim, action_dim).to(device)
        trainer = SuperpositionReplayTrainer(model, lr=config.learning_rate)
    elif approach == 'gates':
        model = GateEnhancedModel(obs_dim, action_dim).to(device)
        trainer = BaselineTrainer(model, lr=config.learning_rate)
    elif approach == 'error_correction':
        model = EnsembleModel(obs_dim, action_dim, num_models=5).to(device)
        trainer = BaselineTrainer(model, lr=config.learning_rate)
    else:
        raise ValueError(f"Unknown approach: {approach}")
    
    # Training
    loss_history = []
    start_time = time.time()
    
    for epoch in range(config.num_epochs):
        epoch_losses = []
        for obs, actions, rewards in batches:
            metrics = trainer.train_step(obs, actions, rewards)
            epoch_losses.append(metrics['loss'])
        loss_history.append(np.mean(epoch_losses))
    
    training_time = time.time() - start_time
    
    # Evaluation
    model.eval()
    with torch.no_grad():
        eval_errors = []
        for obs, actions, rewards in batches[:5]:  # Use subset for eval
            outputs = model(obs, actions)
            error = F.mse_loss(outputs['obs_mean'], obs).item()
            eval_errors.append(error)
    
    return ExperimentResult(
        approach=approach,
        seed=seed,
        final_loss=np.mean(loss_history[-5:]),
        training_time=training_time,
        loss_history=loss_history,
        prediction_error=np.mean(eval_errors)
    )

## 7.4 Run Experiments

In [ ]:
def run_all_experiments(config: ExperimentConfig) -> pd.DataFrame:
    """
    Run all approaches across multiple seeds.
    
    Parameters
    ----------
    config : ExperimentConfig
        Experiment configuration
    
    Returns
    -------
    pd.DataFrame
        Results dataframe
    """
    approaches = ['baseline', 'qaoa', 'superposition', 'gates', 'error_correction']
    
    print(f"Collecting data from {config.env_name}...")
    episodes = collect_data(config.env_name, config.num_episodes)
    
    if not episodes:
        print("No episodes collected!")
        return pd.DataFrame()
    
    batches = create_batches(episodes, config.batch_size, config.seq_len)
    print(f"Created {len(batches)} training batches")
    
    obs_dim = episodes[0]['obs'].shape[1]
    action_dim = episodes[0]['actions'].shape[1]
    
    results = []
    total_runs = len(approaches) * config.num_seeds
    current_run = 0
    
    for approach in approaches:
        for seed in range(config.num_seeds):
            current_run += 1
            print(f"\n[{current_run}/{total_runs}] Running {approach} with seed {seed}...")
            
            result = run_experiment(
                config, approach, seed + 42, batches, obs_dim, action_dim
            )
            results.append(result)
            
            print(f"  Final loss: {result.final_loss:.4f}, "
                  f"Pred error: {result.prediction_error:.4f}, "
                  f"Time: {result.training_time:.2f}s")
    
    # Convert to dataframe
    df = pd.DataFrame([
        {
            'approach': r.approach,
            'seed': r.seed,
            'final_loss': r.final_loss,
            'training_time': r.training_time,
            'prediction_error': r.prediction_error
        }
        for r in results
    ])
    
    return df, results

In [ ]:
# Run experiments on CartPole
config = ExperimentConfig(
    name='cartpole_comparison',
    env_name='CartPole-v1',
    num_seeds=5,
    num_episodes=20,
    num_epochs=50,
    batch_size=32,
    seq_len=20
)

print(f"Starting experiment: {config.name}")
print(f"Environment: {config.env_name}")
print(f"Seeds: {config.num_seeds}, Epochs: {config.num_epochs}")

df, results = run_all_experiments(config)

## 7.5 Statistical Analysis

In [ ]:
def compute_statistics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute summary statistics for each approach.
    
    Parameters
    ----------
    df : pd.DataFrame
        Results dataframe
    
    Returns
    -------
    pd.DataFrame
        Summary statistics
    """
    stats_data = []
    
    for approach in df['approach'].unique():
        approach_df = df[df['approach'] == approach]
        
        for metric in ['final_loss', 'prediction_error', 'training_time']:
            values = approach_df[metric].values
            
            stats_data.append({
                'approach': approach,
                'metric': metric,
                'mean': np.mean(values),
                'std': np.std(values),
                'min': np.min(values),
                'max': np.max(values),
                'ci_lower': np.mean(values) - 1.96 * np.std(values) / np.sqrt(len(values)),
                'ci_upper': np.mean(values) + 1.96 * np.std(values) / np.sqrt(len(values))
            })
    
    return pd.DataFrame(stats_data)

In [ ]:
def compute_pairwise_tests(
    df: pd.DataFrame,
    metric: str = 'final_loss',
    baseline: str = 'baseline'
) -> pd.DataFrame:
    """
    Compute pairwise statistical tests vs baseline.
    
    Parameters
    ----------
    df : pd.DataFrame
        Results dataframe
    metric : str
        Metric to compare
    baseline : str
        Baseline approach name
    
    Returns
    -------
    pd.DataFrame
        Statistical test results
    """
    baseline_values = df[df['approach'] == baseline][metric].values
    
    test_results = []
    
    for approach in df['approach'].unique():
        if approach == baseline:
            continue
        
        approach_values = df[df['approach'] == approach][metric].values
        
        # Mann-Whitney U test
        u_stat, p_value = stats.mannwhitneyu(
            baseline_values, approach_values, alternative='two-sided'
        )
        
        # Cohen's d effect size
        pooled_std = np.sqrt(
            (np.std(baseline_values)**2 + np.std(approach_values)**2) / 2
        )
        cohens_d = (np.mean(baseline_values) - np.mean(approach_values)) / pooled_std
        
        # Effect size interpretation
        if abs(cohens_d) < 0.2:
            effect = 'negligible'
        elif abs(cohens_d) < 0.5:
            effect = 'small'
        elif abs(cohens_d) < 0.8:
            effect = 'medium'
        else:
            effect = 'large'
        
        test_results.append({
            'approach': approach,
            'vs_baseline': baseline,
            'metric': metric,
            'baseline_mean': np.mean(baseline_values),
            'approach_mean': np.mean(approach_values),
            'difference': np.mean(baseline_values) - np.mean(approach_values),
            'u_statistic': u_stat,
            'p_value': p_value,
            'cohens_d': cohens_d,
            'effect_size': effect,
            'significant': p_value < 0.05
        })
    
    return pd.DataFrame(test_results)

In [ ]:
# Compute statistics
summary_stats = compute_statistics(df)

print("Summary Statistics:")
print("="*80)
for metric in ['final_loss', 'prediction_error', 'training_time']:
    print(f"\n{metric.upper()}:")
    metric_df = summary_stats[summary_stats['metric'] == metric]
    for _, row in metric_df.iterrows():
        print(f"  {row['approach']:20s}: {row['mean']:.4f} +/- {row['std']:.4f} "
              f"[{row['ci_lower']:.4f}, {row['ci_upper']:.4f}]")

In [ ]:
# Pairwise tests
loss_tests = compute_pairwise_tests(df, 'final_loss')
error_tests = compute_pairwise_tests(df, 'prediction_error')

print("\nStatistical Tests vs Baseline (Final Loss):")
print("="*80)
for _, row in loss_tests.iterrows():
    sig = "*" if row['significant'] else ""
    print(f"{row['approach']:20s}: d={row['cohens_d']:+.3f} ({row['effect_size']}), "
          f"p={row['p_value']:.4f}{sig}")

print("\nStatistical Tests vs Baseline (Prediction Error):")
print("="*80)
for _, row in error_tests.iterrows():
    sig = "*" if row['significant'] else ""
    print(f"{row['approach']:20s}: d={row['cohens_d']:+.3f} ({row['effect_size']}), "
          f"p={row['p_value']:.4f}{sig}")

## 7.6 Visualizations

In [ ]:
# Learning curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

approach_colors = {
    'baseline': COLORS['baseline'],
    'qaoa': COLORS['qaoa'],
    'superposition': COLORS['superposition'],
    'gates': COLORS['gates'],
    'error_correction': COLORS['error_correction']
}

# Group results by approach
approach_histories = defaultdict(list)
for r in results:
    approach_histories[r.approach].append(r.loss_history)

# Plot learning curves
ax = axes[0]
for approach, histories in approach_histories.items():
    histories_arr = np.array(histories)
    mean_history = histories_arr.mean(axis=0)
    std_history = histories_arr.std(axis=0)
    
    epochs = range(len(mean_history))
    ax.plot(epochs, mean_history, label=approach.replace('_', ' ').title(),
            color=approach_colors[approach], linewidth=2)
    ax.fill_between(epochs, mean_history - std_history, mean_history + std_history,
                   color=approach_colors[approach], alpha=0.2)

ax.set_xlabel('Epoch')
ax.set_ylabel('Training Loss')
ax.set_title('Learning Curves (Mean +/- Std)')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

# Bar plot of final performance
ax = axes[1]
approaches = list(approach_histories.keys())
means = [df[df['approach'] == a]['final_loss'].mean() for a in approaches]
stds = [df[df['approach'] == a]['final_loss'].std() for a in approaches]
colors = [approach_colors[a] for a in approaches]

bars = ax.bar(range(len(approaches)), means, yerr=stds, capsize=5, color=colors, alpha=0.8)
ax.set_xticks(range(len(approaches)))
ax.set_xticklabels([a.replace('_', '\n').title() for a in approaches], fontsize=9)
ax.set_ylabel('Final Loss')
ax.set_title('Final Loss Comparison')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../results/figures/comprehensive_comparison_learning.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Box plots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics = ['final_loss', 'prediction_error', 'training_time']
titles = ['Final Training Loss', 'Prediction Error (MSE)', 'Training Time (s)']

for ax, metric, title in zip(axes, metrics, titles):
    data = [df[df['approach'] == a][metric].values for a in approaches]
    
    bp = ax.boxplot(data, patch_artist=True, labels=[a.replace('_', '\n').title() 
                                                      for a in approaches])
    
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax.set_ylabel(title)
    ax.set_title(title)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../results/figures/comprehensive_comparison_boxplots.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Effect size visualization
fig, ax = plt.subplots(figsize=(10, 6))

# Combine loss and error tests
all_tests = pd.concat([loss_tests, error_tests])

# Create grouped bar chart
x = np.arange(len(loss_tests))
width = 0.35

bars1 = ax.bar(x - width/2, loss_tests['cohens_d'], width, 
               label='Final Loss', color=COLORS['baseline'], alpha=0.8)
bars2 = ax.bar(x + width/2, error_tests['cohens_d'], width,
               label='Prediction Error', color=COLORS['qaoa'], alpha=0.8)

ax.set_ylabel("Cohen's d (vs Baseline)")
ax.set_title('Effect Sizes vs Baseline')
ax.set_xticks(x)
ax.set_xticklabels([a.replace('_', '\n').title() for a in loss_tests['approach']])
ax.legend()
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.axhline(y=0.2, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
ax.axhline(y=-0.2, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
ax.axhline(y=0.5, color='gray', linestyle=':', linewidth=0.5, alpha=0.5)
ax.axhline(y=-0.5, color='gray', linestyle=':', linewidth=0.5, alpha=0.5)
ax.grid(True, alpha=0.3, axis='y')

# Add effect size thresholds
ax.text(ax.get_xlim()[1] + 0.1, 0.2, 'Small', va='center', fontsize=8, color='gray')
ax.text(ax.get_xlim()[1] + 0.1, 0.5, 'Medium', va='center', fontsize=8, color='gray')

plt.tight_layout()
plt.savefig('../results/figures/comprehensive_comparison_effects.png', dpi=150, bbox_inches='tight')
plt.show()

## 7.7 Results Summary Table

In [ ]:
# Create summary table
summary_table = []

for approach in approaches:
    approach_df = df[df['approach'] == approach]
    
    # Get test results
    loss_row = loss_tests[loss_tests['approach'] == approach] if approach != 'baseline' else None
    error_row = error_tests[error_tests['approach'] == approach] if approach != 'baseline' else None
    
    row = {
        'Approach': approach.replace('_', ' ').title(),
        'Final Loss': f"{approach_df['final_loss'].mean():.4f} +/- {approach_df['final_loss'].std():.4f}",
        'Pred. Error': f"{approach_df['prediction_error'].mean():.4f} +/- {approach_df['prediction_error'].std():.4f}",
        'Time (s)': f"{approach_df['training_time'].mean():.1f}"
    }
    
    if loss_row is not None and len(loss_row) > 0:
        sig = '*' if loss_row['significant'].values[0] else ''
        row['vs Baseline'] = f"d={loss_row['cohens_d'].values[0]:+.2f}{sig}"
    else:
        row['vs Baseline'] = '(baseline)'
    
    summary_table.append(row)

summary_df = pd.DataFrame(summary_table)
print("\n" + "="*80)
print("COMPREHENSIVE COMPARISON RESULTS")
print("="*80)
print(f"Environment: {config.env_name}")
print(f"Seeds: {config.num_seeds}, Epochs: {config.num_epochs}")
print("="*80)
print(summary_df.to_string(index=False))
print("="*80)
print("* indicates p < 0.05 vs baseline")

## 7.8 Save Results

In [ ]:
# Save results
results_dir = Path('../results/comparison')
results_dir.mkdir(parents=True, exist_ok=True)

# Save raw results
df.to_csv(results_dir / 'raw_results.csv', index=False)

# Save summary statistics
summary_stats.to_csv(results_dir / 'summary_statistics.csv', index=False)

# Save statistical tests
loss_tests.to_csv(results_dir / 'statistical_tests_loss.csv', index=False)
error_tests.to_csv(results_dir / 'statistical_tests_error.csv', index=False)

# Save summary table
summary_df.to_csv(results_dir / 'summary_table.csv', index=False)

print(f"Results saved to {results_dir}")

## 7.9 Summary

### Key Findings

Based on the comprehensive comparison across multiple seeds:

1. **Training Efficiency**: Compare final loss values and convergence speed
2. **Prediction Quality**: Compare reconstruction errors
3. **Statistical Significance**: Identify which improvements are statistically significant
4. **Effect Sizes**: Understand the practical magnitude of differences

### Conclusions

The results provide evidence for understanding:
- Which quantum-inspired approaches provide benefits
- Under what conditions each approach excels
- The trade-offs between different methods

### Next Steps

- Phase 8: Ablation Studies (component analysis)
- Phase 9: Results & Analysis (final documentation)

In [ ]:
print("\n" + "="*60)
print("Phase 7: Comprehensive Comparison - COMPLETE")
print("="*60)
print("\nCompleted:")
print("  - Unified implementation of all 5 approaches")
print("  - Multi-seed experiments (5 seeds per approach)")
print("  - Statistical analysis (Mann-Whitney U, Cohen's d)")
print("  - 95% confidence intervals")
print("  - Comprehensive visualizations")
print("  - Results saved to results/comparison/")
print("\nReady for Phase 8: Ablation Studies")